In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator().build(number, verbose=False)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count()*5, number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(10000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.998897222140599, 0.9988506000027111)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9995873313809958, 0.9995691131526805)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe(max_rows=10)

In [9]:
y

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
0,ItalianFiscalCode,Error,CadastreCode,Document,Plate,Address,NaN,NaN,Region,Municipality,...,Country,NaN,Name,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,Boolean
1,ItalianFiscalCode,ItalianVAT,NaN,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,NaN,...,Country,CountryCode,Name,NaN,NaN,EMail,PhoneNumber,Date,NaN,Boolean
2,ItalianFiscalCode,ItalianVAT,CadastreCode,Error,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,NaN,NaN,Name,Surname,String,NaN,NaN,Date,BiologicalSex,NaN
3,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Country,CountryCode,Name,NaN,String,NaN,PhoneNumber,Date,NaN,NaN
4,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,NaN,...,Country,CountryCode,Name,Surname,String,EMail,NaN,Date,BiologicalSex,Boolean
5,ItalianFiscalCode,ItalianVAT,NaN,Document,NaN,NaN,NaN,NaN,Region,Municipality,...,Country,Error,Name,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,Boolean
6,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean


In [10]:
model.predict_dataframe(X)

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
0,ItalianFiscalCode,Error,CadastreCode,Document,Plate,Address,NaN,NaN,Region,Municipality,...,Country,NaN,Name,Surname,Error,NaN,PhoneNumber,Date,BiologicalSex,Boolean
1,ItalianFiscalCode,ItalianVAT,NaN,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,NaN,...,Country,CountryCode,Name,NaN,NaN,EMail,PhoneNumber,Date,NaN,Boolean
2,ItalianFiscalCode,ItalianVAT,CadastreCode,Error,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,NaN,NaN,Name,Surname,Error,NaN,NaN,Date,BiologicalSex,NaN
3,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Country,CountryCode,Name,NaN,Error,NaN,PhoneNumber,Date,NaN,NaN
4,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,NaN,...,Country,CountryCode,Name,Surname,Error,EMail,NaN,Date,BiologicalSex,Boolean
5,ItalianFiscalCode,ItalianVAT,NaN,Document,NaN,NaN,NaN,NaN,Region,Municipality,...,Country,Error,Name,Surname,Error,NaN,PhoneNumber,Date,BiologicalSex,Boolean
6,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Country,CountryCode,Name,Surname,Error,EMail,PhoneNumber,Date,BiologicalSex,Boolean


In [11]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('NaN', 'Integer'): 1,
         ('Error', 'Document'): 136,
         ('Country', 'Error'): 6,
         ('String', 'Error'): 62,
         ('CountryCode', 'NaN'): 61,
         ('Error', 'Name'): 5,
         ('Error', 'Country'): 1,
         ('NaN', 'CountryCode'): 90,
         ('Document', 'Error'): 77,
         ('Error', 'ItalianVAT'): 20,
         ('Error', 'CountryCode'): 23,
         ('ItalianZIPCode', 'Error'): 10,
         ('Error', 'String'): 20,
         ('Error', 'NaN'): 6,
         ('Error', 'Float'): 2,
         ('Integer', 'NaN'): 3,
         ('Surname', 'String'): 3,
         ('Name', 'Error'): 24,
         ('Error', 'ItalianZIPCode'): 5,
         ('Error', 'ProvinceCode'): 15,
         ('Date', 'Error'): 3,
         ('Error', 'Region'): 1,
         ('Municipality', 'Error'): 4,
         ('Error', 'ItalianFiscalCode'): 8,
         ('EMail', 'String'): 2,
         ('Error', 'Date'): 1,
         ('CountryCode', 'ProvinceCode'): 7,
         ('NaN', 'Error'): 4,
       